In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from glob import glob
import html2text

## Loading Data

In [2]:
# load data
filenames = glob("data/interim/df_*.csv")
dataframes = [pd.read_csv(f, encoding='latin-1') for f in filenames]

In [3]:
for i in range(17):
    dataframes[i] = dataframes[i].drop('Unnamed: 0', axis=1)

## Some useful functions

In [5]:
# takes a post id and returns a dataframe with that post
def getPost(id):
    
    # define dataframe structure in memory
    df = pd.DataFrame(columns=['by', 'score', 'time', 'timestamp', 'title', 'type', 'url', 'text',
       'parent', 'deleted', 'dead', 'descendants', 'id', 'ranking'])
    
    # find the post and add it to the dataframe in memory
    for dataframe in dataframes:
        df = pd.concat([dataframe[dataframe["id"] == id], df])
        
    return df

In [6]:
# takes a list of post ids and returns the comments on those posts
def getComments(parents, family=None):
    
    # define dataframe structure in memory
    children = pd.DataFrame(columns=['by', 'score', 'time', 'timestamp', 'title', 'type', 'url', 'text',
       'parent', 'deleted', 'dead', 'descendants', 'id', 'ranking'])
    
    # store family
    fam = family
    
    # base case: if no parents provided, return the family
    if not parents:
        return fam
    
    # recursive case: if parents provided, add children to family and check next generation
    else:
        
        # define the next generation
        nextgen = []
        
        # for each dataframe
        for dataframe in dataframes:

            # add the parent's children to the next generation
            children = pd.concat([dataframe[dataframe["parent"].isin(parents)], children]) 
            nextgen += list(children["id"])

            # add the parent's children to the family
            fam = pd.concat([children, fam])

            # reset children for the next parent
            children = pd.DataFrame(columns=['by', 'score', 'time', 'timestamp', 'title', 'type', 'url', 'text',
   'parent', 'deleted', 'dead', 'descendants', 'id', 'ranking'])

        # return getThread on the next generation and current family
        return getComments(nextgen, fam)        

In [7]:
def getThread(id):
    return pd.concat([getPost(id), getComments([id])])

In [8]:
def makeDocument(id):
    return html2text.html2text(" ".join(list(getThread(id)["text"].dropna())))

In [9]:
def imputeDescendants(thread):
    thread = thread.reset_index(drop=True)
    
    thread["count"] = 1
    
    sums = thread.groupby("parent").sum().reset_index()[["parent", "count"]]
    
    descendants = pd.merge(thread, sums, how="left", left_on="id", right_on="parent")
    
    descendants = descendants["count_y"].fillna(0).astype(int)
    
    thread["descendants"] = descendants
    
    return thread

In [10]:
def getMostRecentPostsBy(user):
    url = "https://hacker-news.firebaseio.com/v0/user/" + user + ".json"
    response = requests.get(url, data=data)
    comments = response.json()["submitted"][0:3]
    
    posts = []
    
    for comment in comments:
        url = "https://hacker-news.firebaseio.com/v0/item/" + str(comment) + ".json"
        response = requests.get(url, data=data)
        posts += [response.json()["text"]]
        
    return posts

## Testing the waters with TF IDF

In [11]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df=0, stop_words='english')

In [12]:
# define test documents
sexism = [14988086, 13784509, 14967505]
selfDriving = [15644680, 13718586, 14338328]
augmentedReality = [15124809, 14490239, 14713679]

In [13]:
corpus = [makeDocument(id) for id in sexism + selfDriving + augmentedReality]

In [14]:
# find our most descriptive words!
tfidf_matrix = tf.fit_transform(corpus)
feature_names = tf.get_feature_names()
dense_matrix = tfidf_matrix.todense()
num_documents = len(corpus)
for i in np.arange(num_documents):
    document_words = dense_matrix[i].tolist()[0]
    document_scores = [pair for pair in zip(range(0, len(document_words)), document_words) if pair[1] > 0]
    
    sorted_scores = sorted(document_scores, key=lambda t: t[1] * -1)[0:4] #4 words
    
    print("Top words in document {}".format(i + 1))
    for scores_tuple in sorted_scores:
        print("\tWord: {}, TF-IDF: {}".format(feature_names[scores_tuple[0]], round(scores_tuple[1], 5)))

Top words in document 1
	Word: women, TF-IDF: 0.54592
	Word: men, TF-IDF: 0.25394
	Word: people, TF-IDF: 0.19514
	Word: gender, TF-IDF: 0.15832
Top words in document 2
	Word: sexism, TF-IDF: 0.2909
	Word: manager, TF-IDF: 0.25308
	Word: tank, TF-IDF: 0.23651
	Word: dress, TF-IDF: 0.23221
Top words in document 3
	Word: women, TF-IDF: 0.3793
	Word: people, TF-IDF: 0.24035
	Word: salary, TF-IDF: 0.20347
	Word: memo, TF-IDF: 0.17917
Top words in document 4
	Word: car, TF-IDF: 0.52935
	Word: driving, TF-IDF: 0.29048
	Word: cars, TF-IDF: 0.24951
	Word: waymo, TF-IDF: 0.20291
Top words in document 5
	Word: google, TF-IDF: 0.34588
	Word: uber, TF-IDF: 0.27355
	Word: laptop, TF-IDF: 0.20365
	Word: waymo, TF-IDF: 0.19234
Top words in document 6
	Word: lyft, TF-IDF: 0.51379
	Word: uber, TF-IDF: 0.47412
	Word: driving, TF-IDF: 0.18652
	Word: drivers, TF-IDF: 0.17458
Top words in document 7
	Word: apple, TF-IDF: 0.33966
	Word: ar, TF-IDF: 0.33961
	Word: android, TF-IDF: 0.33788
	Word: ios, TF-IDF: 

In [114]:
imputeDescendants(getThread(14988086)).groupby("by").sum().sort_values(by="descendants", ascending=False).head()

,score,time,descendants,ranking,count
by,,,,,
florianmari,321.0,1.502446e+09,40,NaN,1
imartin2k,NaN,6.009886e+09,19,NaN,4
foldr,NaN,1.953236e+10,12,NaN,13
rayiner,NaN,1.502451e+09,9,NaN,1
scandox,NaN,1.502455e+09,9,NaN,1


In [173]:
getRecentCommentsBy("imartin2k")["text"].dropna()[0]

,by,score,time,timestamp,title,type,url,text,parent,deleted,dead,descendants,id,ranking
1175,imartin2k,1.0,1.521186e+09,2018-03-16 07:42:18 UTC,Can Cambodia stay competitive with so many pub...,story,http://www.phnompenhpost.com/business/can-camb...,NaN,0,NaN,NaN,0.0,16599209,NaN
1178,imartin2k,1.0,1.521105e+09,2018-03-15 09:07:36 UTC,Spotify â?? The Impossible Success Story,story,http://northzone.com/spotify-impossible-succes...,NaN,0,NaN,NaN,0.0,16591317,NaN
359,imartin2k,2.0,1.521100e+09,2018-03-15 07:41:15 UTC,Lessons of History: summary in bullets,story,https://medium.com/@yanda/lessons-of-history-s...,NaN,0,NaN,NaN,0.0,16591039,NaN
173,imartin2k,3.0,1.521098e+09,2018-03-15 07:20:31 UTC,Icelandâ??s male circumcision ban,story,http://www.independent.co.uk/news/world/europe...,NaN,0,NaN,NaN,0.0,16590979,NaN
703,imartin2k,1.0,1.521019e+09,2018-03-14 09:20:38 UTC,Why Blockchain Liquidity Is a Bad Thingâ?¦,story,https://hackernoon.com/liquidity-kind-of-sucks...,NaN,0,NaN,NaN,0.0,16583391,NaN
708,imartin2k,1.0,1.521019e+09,2018-03-14 09:11:48 UTC,The Questions to Ask About the News,story,http://scottberkun.com/2018/the-questions-to-a...,NaN,0,NaN,NaN,0.0,16583352,NaN
1228,imartin2k,1.0,1.520923e+09,2018-03-13 06:39:53 UTC,What happened to 90â??s optimism? Did Silicon ...,story,https://roadlesstravelled.me/2018/03/13/what-h...,NaN,0,NaN,NaN,0.0,16574217,NaN
362,imartin2k,1.0,1.520829e+09,2018-03-12 04:36:40 UTC,You Think You Want Media Literacyâ?¦ Do You?,story,https://points.datasociety.net/you-think-you-w...,NaN,0,NaN,NaN,0.0,16566240,NaN
1173,imartin2k,3.0,1.520820e+09,2018-03-12 01:56:57 UTC,The Man Who Knew Too Little,story,https://www.nytimes.com/2018/03/10/style/the-m...,NaN,0,NaN,NaN,0.0,16565744,NaN
1182,imartin2k,97.0,1.520057e+09,2018-03-03 05:56:38 UTC,Hacking the Brain with Adversarial Images,story,https://spectrum.ieee.org/the-human-os/robotic...,NaN,0,NaN,NaN,29.0,16508139,NaN


In [174]:
im = getRecentCommentsBy("imartin2k")

In [189]:
im[im["type"] == "comment"]

,by,score,time,timestamp,title,type,url,text,parent,deleted,dead,descendants,id,ranking
242,imartin2k,NaN,1.519230e+09,2018-02-21 16:16:21 UTC,NaN,comment,NaN,"Fittingly, from the recent NYT WeWork feature:...",16430259,NaN,NaN,NaN,16430512,NaN
249,imartin2k,NaN,1.518987e+09,2018-02-18 20:43:16 UTC,NaN,comment,NaN,There is a so called Gender Inequality Index b...,16408353,NaN,NaN,NaN,16408434,NaN
92,imartin2k,NaN,1.518866e+09,2018-02-17 11:09:05 UTC,NaN,comment,NaN,Itâ??s actually a bit more complicated. The Is...,16400101,NaN,NaN,NaN,16400171,NaN
1126,imartin2k,NaN,1.518708e+09,2018-02-15 15:19:34 UTC,NaN,comment,NaN,"""Yes, neither car mechanics nor pre-school tea...",16383132,NaN,NaN,NaN,16384665,NaN
1139,imartin2k,NaN,1.518689e+09,2018-02-15 10:08:18 UTC,NaN,comment,NaN,Neither. I only was critizing what I think is ...,16382916,NaN,NaN,NaN,16382934,NaN
317,imartin2k,NaN,1.518689e+09,2018-02-15 09:59:01 UTC,NaN,comment,NaN,Itâ??s important to investigate this topic fro...,16382605,NaN,NaN,NaN,16382894,NaN
667,imartin2k,NaN,1.518114e+09,2018-02-08 18:25:02 UTC,NaN,comment,NaN,Very nice one @ exolymph.\r\n\r\n,16334181,NaN,NaN,NaN,16334297,NaN
309,imartin2k,NaN,1.518107e+09,2018-02-08 16:30:07 UTC,NaN,comment,NaN,Systematic outrage and Trump can be successful...,16331886,NaN,NaN,NaN,16333322,NaN
669,imartin2k,NaN,1.517996e+09,2018-02-07 09:29:39 UTC,NaN,comment,NaN,"In my eyes, this is a case where no matter fro...",16322704,NaN,NaN,NaN,16323250,NaN
781,imartin2k,NaN,1.517861e+09,2018-02-05 19:59:05 UTC,NaN,comment,NaN,I guess this situation will only permanently c...,16311192,NaN,NaN,NaN,16311546,NaN


nan

In [129]:
def getRecentCommentsBy(user):
    empty = pd.DataFrame(columns=['by', 'score', 'time', 'timestamp', 'title', 'type', 'url', 'text',
       'parent', 'deleted', 'dead', 'descendants', 'id', 'ranking'])
    for dataframe in dataframes:
        empty = pd.concat([dataframe[dataframe["by"] == user], empty])  
    
    return empty.reset_index(drop=True).sort_values(by="timestamp", ascending=False)

In [234]:
getMostRecentPostsBy("raymondgh")

['Perfect. I&#x27;ll send this with my cover letter next time :D',
 'What does it do? I see benefits but I don&#x27;t understand how the app will help me achieve those.',
 'Our language is evolving back to hieroglyphics']

In [224]:
url = "https://hacker-news.firebaseio.com/v0/item/" + str(16651855) + ".json"
response = requests.get(url, data=data)
response.json()["text"]

'Perfect. I&#x27;ll send this with my cover letter next time :D'

In [227]:
a + response.json()["text"]

TypeError: can only concatenate list (not "str") to list

In [205]:
a.append(a)

In [206]:
a

[1, 2, 3, [...]]

In [210]:
a = [1, 2, 3]

In [197]:
import requests
url = 'https://hacker-news.firebaseio.com/v0/item/16430512.json'
data = 0
response = requests.get(url, data=data)

[16651855, 16580264, 16571490]

In [195]:
from html2text import html2text

In [196]:
html2text(response.json()['text'])

'Fittingly, from the recent NYT WeWork feature: "More than most companies,\nWeWork promotes the consumption of alcohol as an inherent virtue. "\n[https://www.nytimes.com/2018/02/17/business/the-wework-\nmanif...](https://www.nytimes.com/2018/02/17/business/the-wework-manifesto-\nfirst-office-space-next-the-world.html)\n\n'

In [171]:
import html2text
h = html2text.HTML2Text()
h.ignore_links = True
print( h.handle("<p>Hello, <a href='http://earth.google.com/'>world</a>!"))

Hello, world!




In [18]:
dataframes[0][dataframes[0]["score"] == 3001]

,by,score,time,timestamp,title,type,url,text,parent,deleted,dead,descendants,id,ranking
497622,vladikoff,3001.0,1.511898e+09,2017-11-28 19:41:10 UTC,macOS High Sierra: Anyone can login as â??root...,story,https://twitter.com/lemiorhan/status/935578694...,NaN,0,NaN,NaN,1055.0,15800676,NaN
